In [1]:
import networkx as nx
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
from dask.distributed import Client, progress
import dask
client = Client()
client

Client Scheduler: tcp://127.0.0.1:58846 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [3]:
#################################################################################
############################### Helper Functions  ###############################
#################################################################################

def set_influence(G, value, node=None):
    '''
        Set influence of a node in a network G or
        set influence of all nodes to value.
        G      ::  a networkx graph
        node   ::  a reference to a node in G
        value  ::  an integer 0 or 1
    '''
    if node:
        G.nodes[node][label] = value
    else:
        influence_attrib = { i : value for i in range(N) }
        nx.set_node_attributes(G,influence_attrib, label)
        
def get_is_influenced(G, node):
    return G.nodes[node][label]
        
def get_number_influenced(G):
    '''
        Get the number of influenced nodes.
    '''
    return sum(nx.get_node_attributes(G, label).values())

def check_can_influence(G, node, q):
    '''
        Determines whether a node is influenced by
        its neighbours. (Threshold Check)
    '''
    if get_is_influenced(G, node) == 1:
        return False
    
    friends = list(G.neighbors(node))
    num_friends = len(friends)
    
    if num_friends == 0:
        return False
    
    num_influenced = sum([1 for friend in friends if G.nodes[friend][label] == 1])
    
    if num_influenced/num_friends > q:
        return True
    return False

#################################################################################
########################## Simulation Helper Functions ##########################
#################################################################################

def spread_influence(G, current_node, phi):
    '''
        Recursive function to spread influence 
        from current_node in G.
    '''
    
    ## Not interesting
    if get_is_influenced(G, current_node) == 0 and not check_can_influence(G, current_node, phi):
        return
    else:
        set_influence(G, 1, current_node)
        ## Find uninfluenced friends
        friends = list(G.neighbors(current_node))
        targets = [friend for friend in friends if G.nodes[friend][label] == 0]
        
        for friend in targets:
            spread_influence(G, friend, phi)
        
def get_vulnerable(G, phi):
    '''
        Check for vulnerable nodes.
        Return list of vulnerable nodes.
    '''
    vulnerable = []
    for node in G.nodes():
        if check_can_influence(G, node, phi):
            vulnerable.append(node)
    return vulnerable
        
def simulate_spread(G, nodes, phi):
    '''
        Simulates the spread of influence starting from each 
        node in nodes and returns a list containing the 
        number of influenced from starting at each node.
    '''
    S = []
    for node in nodes:
        G_tmp = G.copy()
        set_influence(G_tmp, 1, node)
        spread_influence(G_tmp, node, phi)
        
        vulnerable = get_vulnerable(G_tmp, phi)

        while len(vulnerable) != 0:
            for vul in vulnerable:
                spread_influence(G_tmp, vul, phi)
            vulnerable = get_vulnerable(G_tmp, phi)

        S.append(get_number_influenced(G_tmp))
        
    return S

In [4]:
def run_simulation(probability):
    G = nx.erdos_renyi_graph(N, probability)
    set_influence(G, 0)
    ## Retrieve influential nodes - top q% and non-influential nodes
    degree_ordered_nodes = sorted(list(G.nodes()), key=lambda x: G.degree(x), reverse=True)
    influential_nodes = degree_ordered_nodes[:int(q*N)]
    normal_nodes = degree_ordered_nodes[int(q*N):]
    ## Simulation
    influential = simulate_spread(G, influential_nodes, phi)
    normal = simulate_spread(G, normal_nodes, phi)
    ## Store results
    S_influential = np.mean(influential)
    S_normal = (np.mean(normal))
    
    return [S_influential, S_normal]

In [5]:
## Parameters

N = 250
q = 0.1   
phi = 0.18
max_n_avg = 4
increment = 0.2
num_simulations = 5
label = 'is_influenced'

n_avg = np.arange(1, max_n_avg, increment)
p = [avg/(N-1) for avg in n_avg]
n = len(p)
# S = np.zeros([num_simulations, n, 2])
S = []

In [ ]:
%%time

S = []
for i in tqdm(range(num_simulations)):
    for j in range(n):
        S.append(dask.delayed(run_simulation)(p[j]))

re = dask.compute(S)

In [ ]:
%%time

S = []
for i in tqdm(range(num_simulations)):
    for j in range(n):
        S.append(dask.delayed(run_simulation)(p[j]))

re = dask.compute(S)

In [6]:
%%time

S = []
S.append(client.map(run_simulation,p*num_simulations))
results = client.gather(S)

results

CPU times: user 387 ms, sys: 60.6 ms, total: 448 ms
Wall time: 8.41 s


[[[48.84, 17.093333333333334],
  [105.32, 49.48888888888889],
  [59.96, 25.373333333333335],
  [123.68, 75.55111111111111],
  [170.36, 122.36444444444444],
  [240.2, 166.22222222222223],
  [199.0, 155.4488888888889],
  [250.0, 217.93333333333334],
  [250.0, 214.63111111111112],
  [214.0, 174.63111111111112],
  [250.0, 215.78222222222223],
  [250.0, 210.26666666666668],
  [250.0, 220.1822222222222],
  [250.0, 223.45777777777778],
  [250.0, 224.56444444444443],
  [48.84, 17.093333333333334],
  [105.32, 49.48888888888889],
  [59.96, 25.373333333333335],
  [123.68, 75.55111111111111],
  [170.36, 122.36444444444444],
  [240.2, 166.22222222222223],
  [199.0, 155.4488888888889],
  [250.0, 217.93333333333334],
  [250.0, 214.63111111111112],
  [214.0, 174.63111111111112],
  [250.0, 215.78222222222223],
  [250.0, 210.26666666666668],
  [250.0, 220.1822222222222],
  [250.0, 223.45777777777778],
  [250.0, 224.56444444444443],
  [48.84, 17.093333333333334],
  [105.32, 49.48888888888889],
  [59.96, 

In [ ]:
S


In [ ]:
import dask.bag as db

In [ ]:
b = db.from_sequence(p*num_simulations, npartitions=num_simulations)


In [ ]:
%%time
l = b.map(run_simulation).compute()

In [ ]:
S_influential = S[:,:,0]
S_normal = S[:,:,1]

In [ ]:
##########################
### Processing Results ###
##########################
names = ["Influential", "Normal"]

# Number of Nodes of Network Influenced
N_influential = np.apply_along_axis(np.mean, 0, S_influential)
N_normal = np.apply_along_axis(np.mean, 0, S_normal)

# Proportion of Network Influenced
P_influential = [x/N for x in N_influential]
P_normal = [x/N for x in N_normal]

In [ ]:
############
### Pots ###
############

plt.plot(n_avg, N_influential, "-s")
plt.plot(n_avg, N_normal, "-o")
plt.ylabel("Average Number Influenced")
plt.xlabel("Average Degree")
plt.title("Number of Nodes Influenced")
plt.legend(names)

In [ ]:
plt.plot(n_avg, P_influential, "-o")
plt.plot(n_avg, P_normal, "-o")
plt.ylabel("Average Number Influenced")
plt.xlabel("Average Degree")
plt.title("Percentage of Network Influenced")
plt.legend(names)